<a href="https://colab.research.google.com/github/davidwhogg/Sailing/blob/main/ipynb/sailing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anisotropic ram-pressure sailing model

## to-do items:
- do all the things in this notebook labeled HOGG.
- Make the optimal sail-setting and route-planning use lookup tables.
- (maybe) Use jax-numpy instead of numpy to get all derivatives?
- Solve the problem of setting the sail *best*, given the keel orientation.
- Solve the problem of setting both the sail and keel best if you want to go in a particular direction.
- Solve the problem of maximizing the magnitude of the velocity.
- Plot things in the air rest frame too.
- Fix matplotlib arrow drawing, which is **all wrong**.


In [ ]:
import numpy as np
import pylab as plt
import scipy.optimize as op
np.random.seed(17)

In [ ]:
# set the fixed properties of the world and boat

rho_water = 1.0 # kg / m^3
rho_air = 0.0014 # kg / m^3
A_perp_k = 1.0 # m^2
A_perp_s = A_perp_k * rho_water / rho_air # seems like sensible design
A_par_k = 0.01 * A_perp_k # very optimistic
A_par_s = 0.01 * A_perp_s # very optimistic

In [ ]:
# set the variable properties of the world

vair = np.array([10., 0.]) # m / s
vwater = np.array([0., 0.]) # m / s water rest frame

In [ ]:
# define functions to make tensors

def A(Aperp, Apar, theta):
  ct = np.cos(theta)
  st = np.sin(theta)
  eperp = np.array([ct, st])
  epar = np.array([-st, ct])
  return Aperp * np.outer(eperp, eperp) + Apar * np.outer(epar, epar)

def A_air(theta):
  return A(A_perp_s, A_par_s, theta)

def A_water(theta):
  return A(A_perp_k, A_par_k, theta)

In [ ]:
# force
TINY = 1.e-8

def _force(rho, dv, AA):
  absdv = np.sqrt(dv @ dv) + TINY
  return rho * absdv * AA @ dv, \
      - (rho / absdv) * AA @ np.outer(dv, dv) - rho * absdv * AA

def force(vboat, vair, vwater, theta_s, theta_k):
  fa, dfadv = _force(rho_air, vair - vboat, A_air(theta_s))
  fw, dfwdv = _force(rho_water, vwater - vboat, A_water(theta_k))
  return fa + fw, dfadv + dfwdv

In [ ]:
# use physics to find the steady state

def get_vboat(vair, vwater, theta_s, theta_k, maxiter=100_000):
  vbig = 10. * np.sqrt(np.sum((vair - vwater) ** 2))
  vb = np.array([0., 0.])
  ff, dfdv = force(vb, vair, vwater, theta_s, theta_k)
  iter = 0
  while (ff @ ff) > 1.e-10 and iter < maxiter and np.all(vb < vbig):
    # print(iter, vb, ff, dfdv, np.linalg.solve(dfdv, ff))
    vb -= np.linalg.solve(dfdv, ff)
    ff, dfdv = force(vb, vair, vwater, theta_s, theta_k)
    iter += 1
  if iter >= maxiter:
    print("get_vboat(): WARNING: Terminated on maxiter.")
  if np.any(vb > vbig):
    print("get_vboat(): WARNING: Terminated on large velocity.")
  return vb
  
# THIS VERSION IS STUPID
def old_get_vboat(vair, vwater, theta_s, theta_k, foo=1.e-4, maxiter=100_000):
  vbig = 10. * np.sqrt(np.sum((vair - vwater) ** 2))
  vb = np.array([0., 0.])
  ff, dfdv = force(vb, vair, vwater, theta_s, theta_k)
  iter = 0
  while (ff @ ff) > 1.e-8 and iter < maxiter and np.all(vb < vbig):
    vb += foo * ff
    ff, dfdv = force(vb, vair, vwater, theta_s, theta_k)
    iter += 1
  if iter >= maxiter:
    print("old_get_vboat(): WARNING: Terminated on maxiter.")
  if np.any(vb > vbig):
    print("old_get_vboat(): WARNING: Terminated on large velocity.")
  return vb

In [ ]:
# testing

theta_k = 0.
theta_s = np.pi / 7.
vb = [0., 0.]
print(force(vb, vair, vwater, theta_s, theta_k))
vboat = get_vboat(vair, vwater, theta_s, theta_k)
print(theta_k, theta_s, vboat)

In [ ]:
# viz

def plot_boat(vair, vwater, theta_s, theta_k):
  foo = -2.
  plt.arrow(foo * vair[0], foo * vair[1], vair[0], vair[1], head_width=0.75, color="k")
  plt.text(foo * vair[0], foo * vair[1] + 0.1 * vair[0], "wind")
  seigv = np.linalg.eigh(A_air(theta_s))[1][0]
  plt.plot([-7. * seigv[0], 7. * seigv[0]], [-6. * seigv[1], 6. * seigv[1]], "k-", lw=1.)
  keigv = np.linalg.eigh(A_water(theta_k))[1][0]
  plt.plot([-4. * keigv[0], 4. * keigv[0]], [-5. * keigv[1], 5. * keigv[1]], "k-", lw=6., alpha=0.2)
  vboat = get_vboat(vair, vwater, theta_s, theta_k)
  plt.arrow(0., 0., vboat[0], vboat[1], head_width=0.75, color="k")
  plt.xlim(-24, 24)
  plt.ylim(-24, 24)
  plt.gca().set_aspect("equal")

In [ ]:
plot_boat(vair, vwater, theta_s, theta_k)

In [ ]:
# 5 random cases of sailing non-optimally

for theta_s, theta_k in 2. * np.pi * np.random.uniform(size=(5, 2)):
  plt.figure()
  plot_boat(vair, vwater, theta_s, theta_k)

In [ ]:
# a few wind-aligned cases

for theta_s in (0., np.pi / 2.):
  for theta_k in (0., np.pi / 2.):
    plt.figure()
    plot_boat(vair, vwater, theta_s, theta_k)

In [ ]:
# we could do here a whole grid of options, for the paper.... HOGG DO THAT

In [ ]:
# now do a very large number of trials:
ntrials = 3000
vavbmax, vavbmin = 0., 0.
vboats = np.zeros((ntrials, 2))
for trial in range(ntrials):
  theta_s, theta_k = 2. * np.pi * np.random.uniform(size=(2))
  vb = get_vboat(vair, vwater, theta_s, theta_k)
  vboats[trial] = vb
  if vair @ vb > vavbmax:
    vavbmax = vair @ vb
    downwind = (theta_s, theta_k)
  if vair @ vb < vavbmin:
    vavbmin = vair @ vb
    upwind = (theta_s, theta_k)

In [ ]:
# HOGG: IS IT A COINCIDENCE THAT THIS LOOKS LIKE a projection of Y_10(theta, phi)?

plt.axvline(vair[0], color="k", alpha=0.25, lw=1) # WARNING: BRITTLE
plt.axvline(-vair[0], color="k", alpha=0.25, lw=1) # WARNING: BRITTLE
plt.plot(vboats[:, 0], vboats[:, 1], "k.", ms=1)
plt.plot(vboats[:, 0], -vboats[:, 1], "k.", ms=1) # WARNING: BRITTLE
plt.xlim(-24, 24)
plt.ylim(-24, 24)
plt.gca().set_aspect("equal")
plt.xlabel("x-velocity of boat")
plt.ylabel("y-velocity of boat")
plt.title("lots of random sail-keel settings")

In [ ]:
plot_boat(vair, vwater, *downwind)
plt.figure()
plot_boat(vair, vwater, *upwind)

In [ ]:
# now consier best-possible boat settings, given a vector towards the destination

# HOGG WARNING: THIS CODE IS NOT CORRECT BECAUSE IT DOES LOCAL OPTIMIZATION!
# I THINK WE NEED TO GO TO INTERPOLATION OF A LOOKUP TABLE.
def get_best_thetas(r_dest, vair, vwater):
  def foo(pars):
    return -r_dest @ get_vboat(vair, vwater, *pars)
  theta_s0 = np.arctan2(r_dest[1], r_dest[0])
  theta_k0 = theta_k + 0.5 * np.pi
  res = op.minimize(foo, (theta_s0, theta_k0), method="Nelder-Mead")
  return res['x']

In [ ]:
r_dest = np.array([2., 1.])
best = get_best_thetas(r_dest, vair, vwater)
plot_boat(vair, vwater, *best)

In [ ]:
# now plot a course from A to B
# HOGG: WRONG: SEE ABOVE

def make_course(r_A, r_B, vair, vwater, dt = 0.1):
  bar = (r_B - r_A) @ (r_B - r_A)
  rr = 1. * r_A
  rrs = [rr, ]
  r_AB = r_B - rr
  while (rr - r_A) @ (r_B - r_A) < bar:
    best = get_best_thetas(r_AB, vair, vwater)
    vb = get_vboat(vair, vwater, *best)
    rr = rr + vb * dt
    r_AB = r_B - rr
    rrs.append(rr)
  return np.array(rrs)

In [ ]:
rA = np.array([0., 0.])
rB = np.array([-100., 100.])
rrs = make_course(rA, rB, vair, vwater)

In [ ]:
print(rrs.shape)
plt.plot([rA[0],], [rA[1],], "ko")
plt.plot([rB[0],], [rB[1],], "ro")
plt.plot(rrs[:,0], rrs[:,1], "k-")
plt.plot(rrs[:,0], rrs[:,1], "k.")
plt.gca().set_aspect("equal")
plt.xlabel("x")
plt.ylabel("y")
plt.title("this plot is wrong")